# Convolutinal Neural Network Implementation with the MNIST Dataset
The MNIST dataset contains many samples, and is therefore ideal to implement a neural network on. I will be using a convolutional neural network (CNN from now on) since the samples are images. The MNIST dataset can be found here: https://www.kaggle.com/c/digit-recognizer/data. Note that the data I use in this notebook is only from the "train" set. This is because the test set does not contain labels for its samples, and is therefore only useful for participating in the Kaggle competition "Digit Recognizer". 

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD

Using TensorFlow backend.


## Load Data

In [2]:
data = pd.read_csv('MNIST.csv')

data.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

## Data Exploration

Firstly, I check the distribtuion of the label values to ensure that my model will be trained and evaluated on data containing a fair number of samples for each digit. This ensures that the model will be able to generalise the distinctions between each digit effectively.

In [3]:
data['label'].value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

There are a similar amount of samples for each digit, so we can proceed without having to manipulate the data.

Next, I am going to check for any missing values.

In [4]:
data.isnull().sum().describe()

count    785.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64

Great, a dataset without any missing values is a rarity, so best appreciate the moment...

## Data Preprocessing

### Scale
From the description of the dataset, it is known that the pixels take on values ranging from 0 to 255. For neural networks, it is best practice to scale the data to be in the range of 0 to 1 inclusive. This ensures that the model does not over/under estimate the weights of the higher/lower values, and it is also a bonus that the model converges faster.    

In [5]:
features = data.drop('label', 1)
featutes = features / 255

label = data['label']

### Reshape
I reshape the data in the form (n samples, 28, 28, 1). The first input corresponds to the number of samples in the data, while the next the next three represent the dimensions of the samples. The first two dimensions represent the image of the digits, while the third dimenson represents the number of colour channels - in this case the images are grayscale, so there is only one channel.

In [6]:
features = features.values.reshape(-1, 28, 28, 1)

### Label Encoding
I encode the labels as one-hot vectors. This ensures each class is represented as either on or off. This is better than labelling the digits by their actual values, as this would represent an ordinal class, i.e., the class "3" is the same as the class "9", but scaled down by a factor of three. This is preposterous, as it would suggest that an image of the class "3" could be converted into an image of a 9 just by multiplying the image's features by a factor of three.

In [7]:
label = to_categorical(label, 10)

### Split the Dataset
I split the data set into a train and validation set.

In [8]:
x_train, x_val, y_train, y_val = train_test_split(features, label, test_size=.2)

## Modelling

### Model Architecture
I will be using Keras's Sequential API to build the CNN. This API allows me to add layers to the network as I go. 

I have searched the web for a method of building the perfect CNN, but this seems to be a pipe dream. Most articles I read agree that the best way to finding the optimal architecture is simply trial and error. However, there is a simple framework I will be using that looks like this:

Image Pixels -> Convolutional Layer -> Pooling Layer -> Dropout Layer -> Flatten Layer -> Dense Layer -> Softmax Layer 

The "Convolutional Layer" detects the images' features, and so more complex images might require two or three layers before being passed on to the poolng layer.

The "Pooling Layer" takes the features learned from the "Convolutional Layer" and compresses them by selecting the most important features within a given area. This layer's purpose is reduce the dimensionality of the features, and therefore reduce the computational cost of the network.

The "Dropout Layer" randomly sets the weights of a selected percentage of nodes to zero. This forces the network to distribute the weights more evenly, and therefore reduce overfitting.

The "Flatten Layer" converts the output of the previous layer into a 1D vector. This is useful when wanting to use the output as input for a "Dense Layer".

The "Dense Layer" is the classic hidden layer in a neural network. Its number of neurons is defined by the user. This layer will try to determine the relationships between the outputs of the previous layer.

The "Softmax Layer" is a "Dense Layer" with 10 neurons - one for each class. It is this layer which predicts the class of a sample.

The architecture of a CNN can vary depending on the task. Common variations of the architecture are:

To place more than one "Convolutional Layer" in a row. This is necessary when an image is large and/or complex.

To group a number of layers together, and then use this new group again. This is helpful if you would want to include three "Pooling Layers" in your network ,for example, but you can't just have them one after another, so instead you put them into three different groups, and then place the groups one after another. An example of a group is:

"Convolutional Layer" -> "Pooling Layer" -> "Dropout Layer"

### Hyperparameters
Lastly before building the model, I would like to explain how I would choose the hyperparameters for the layers in the model.

Activation function - I will be using the ReLU activation function for all layers except for the final layer. ReLU is the default activaton function for most modern neural networks. It outputs max(0, x) of the input, and so is a non-linear function which allows it to capture non-linear relationships between the inputs. It is for this reason and because it is so simple that it is the activation function of choice. The final layer will use a softmax activation function. The softmax function takes as input the output of the previous layer, and converts the input into probabilities, which each represent the 10 digit classes. The class with the highest probability is selected.

Filters - The number of seperate windows to go over the input. Having many seperate windows allows for many different starting weights for the input, which should make it easier for the network to find the optimum weights for the input. As far as I can tell, the way in which this hyperparameter is configured is by going up in powers of two, and then pick the best performing number after a few configurations have been tried.

Kernel size - The kernel size is the span of the window that moves across the image. This window captures important related features. It does this by weighting each pixel within the window, and taking their sum. So, the window size reflects the importance of the relationship between the pixels in the window.. So, less complicated images such as digits would require a smaller window size than that of images of pieces of art for example.

Padding - This is the dimensionality of the output after striding across the input. I will use padding='same', which outputs the same dimensions as the input. When the output is reduced in dimensionality, we say that 'valid padding' has been applied. This would be useful for images with redundant information on their edges, which for the images used in this dataset, is not true. 

Pool size - Used as the kernel size for the pooling layer. Functions the same as the kernel size, but will be smaller than the kernel size, so that the pooling layer does not loose the generality of the image. The pooling window will take the maximum activation within the window, and only return that. So, you can imagine that if the pooling window size were too large, we would be reducing the dimensionality of the image too much.

Dropout rate - Chooses the percentage of nodes to leave out when training. From what I can gather, earlier layers favour dropout rates of about 25%, while the final layer favours a rate of about 50%. This makes sense as we do not want to leave too many nodes out early on, as the network as a whole does not have a good representation of the input yet. But, the higher dropout rate works for the final dropout layer because most of the learned features from the input has been learned by this point.

Hidden layer size - Applicable for the "Dense layer". This number represents the number of different features associated with the input. So if the input is a complex image, containing many important interactions between the pixels, then the dense layer will favour a high number of hidden neurons. As far as I can gather from the web, the increments to test are in powers of two.

### Building the Model
After some trial and error, I have found an architecture that works well. The process of trial and error that I used was simply to try out different architectures, and see how well they peformed once trained. The score evaluation code will be shown later on. I have left out showing my testing of the poorer performing architectures, as I feel it would be tedious for the reader to go through them all.

Once I had established the architecture I would be using, I then did the same tiral and error for the hyperparameters.

In [9]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=.25))

model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=.5))
model.add(Dense(units=10, activation='softmax'))

You may have noticed that although the type and ordering of the layers in the first and second blocks of code are the same, two of their hyperparameters are different. The "filters" parameter of the "Conv2D" layers in the second block of code have increased from 32 to 64. This makes intuitive sense to me, since the filters offset the randomness of the weight intialisation, and so the second block's input is more random in terms of importance of the pixels of the image. This is to be expected, as the images recieved by the second block have already been transformed by the first block, and so the new images represent higher level features, resulting in more complex and seemingly almost random features. This, in my mind, is also the reason why the kernel size has been decreased.

The number of "units" in the second from last "Dense" layer is 256, which may seem arbitrary, but, like other parameter inputs, is a power of two. So, I increased the power of two for this layer until the performance of my model diminished to a negligible increase in performance. 

### Optimizer
An optimizer simply determines how we will go about updating our parameters. Optimizers, much like architectures, do not have set use cases. So, similarly to how I chose the architecture of the model, I will also try many different optimizers, and choose the best performing one. The optimizers I will be trying will be the ones provided by the "optimizers" package of the keras library. From my trials, the SGD optimizer performed best, and so it will be the optimizer I choose.

In [10]:
optimizer = SGD()

### Compiling
Finally before training, we need to configure the model. We will be using the Categorical Crossentropy loss function, as it is the default when using a softmax output layer. Finally, we will be evaluating our model on accuracy, as we want to correctly predict as many images as possible.

In [11]:
model.compile(optimizer=optimizer , loss="categorical_crossentropy", metrics=["accuracy"])

### Annealing
Finally, we use a Keras callback called ReduceLROnPLateau. This callback simply reduces the learning rate when our chosen metric has stopped improving. In code, it looks like this:

In [12]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy')

### Evaluating the model
Our model is set-up, so it's time to evaluate it.

In [13]:
epochs = 5
batch_size = 86

model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val), callbacks=[reduce_lr])

Train on 33600 samples, validate on 8400 samples
Epoch 1/5
33600/33600 [==============================] - 112s 3ms/step - loss: 0.9175 - accuracy: 0.7290 - val_loss: 0.1723 - val_accuracy: 0.9446
Epoch 2/5
33600/33600 [==============================] - 122s 4ms/step - loss: 0.2372 - accuracy: 0.9270 - val_loss: 0.1094 - val_accuracy: 0.9640
Epoch 3/5
33600/33600 [==============================] - 123s 4ms/step - loss: 0.1677 - accuracy: 0.9477 - val_loss: 0.1204 - val_accuracy: 0.9643
Epoch 4/5
33600/33600 [==============================] - 124s 4ms/step - loss: 0.1402 - accuracy: 0.9571 - val_loss: 0.0772 - val_accuracy: 0.9745
Epoch 5/5
33600/33600 [==============================] - 125s 4ms/step - loss: 0.1191 - accuracy: 0.9629 - val_loss: 0.0722 - val_accuracy: 0.9760


Setting epochs to around 30 yields the best results, but for the purpose of this note book, 5 epochs is good enough to show the effectiveness of the model, as is evident by the validation accuracy! 